In [1]:
import os
import random
import shutil


def split_dataset(source_dir, output_dir, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    """
    Splits each image class in the source_dir into train, validation, and test directories.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    train_dir = os.path.join(output_dir, "train")
    val_dir = os.path.join(output_dir, "validation")
    test_dir = os.path.join(output_dir, "test")

    for dir_path in [train_dir, val_dir, test_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    classes = [d for d in os.listdir(
        source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    for class_name in classes:
        class_src = os.path.join(source_dir, class_name)
        images = [f for f in os.listdir(
            class_src) if os.path.isfile(os.path.join(class_src, f))]
        random.shuffle(images)

        total = len(images)
        train_end = int(total * train_ratio)
        val_end = train_end + int(total * val_ratio)

        splits = {
            "train": images[:train_end],
            "validation": images[train_end:val_end],
            "test": images[val_end:]
        }

        for split, file_list in splits.items():
            class_dst = os.path.join(output_dir, split, class_name)
            if not os.path.exists(class_dst):
                os.makedirs(class_dst)
            for file_name in file_list:
                src_file = os.path.join(class_src, file_name)
                dst_file = os.path.join(class_dst, file_name)
                shutil.copy2(src_file, dst_file)
    print(f"Dataset split into train, validation, and test in '{output_dir}'.")


# Paths to dataset directory
Base_Folder = 'D:/Learning/University of sadat/Grade 4/Semester 2/06- Graduation Project/Coding/'
original_dataset_dir = f'{Base_Folder}00- The DataSet/00- Dogs Femur Fracture'
split_dataset_dir = f'{Base_Folder}00- The DataSet/Dataset_split_Before_Preprocessing'
split_dataset(original_dataset_dir, split_dataset_dir)

Dataset split into train, validation, and test in 'D:/Learning/University of sadat/Grade 4/Semester 2/06- Graduation Project/Coding/00- The DataSet/Dataset_split_Before_Preprocessing'.


In [4]:
from keras_flops import get_flops  # For FLOPs calculation
import shutil
import time
import math
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import backend as K

from tensorflow.keras.applications import VGG19, ResNet50, VGG16, MobileNetV2, Xception, EfficientNetB0, DenseNet121

from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnetb0_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet121_preprocess

from tensorflow.keras.layers import InputLayer

from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, roc_curve, auc, classification_report,roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
import pickle
import datetime


In [ ]:


def extract_features(generator, model, number_of_batches):
    """
    Extract features from a dataset using a feature extractor model.

    Args:
        generator (ImageDataGenerator): A generator that yields batches of images and labels.
        model (Model): A pre-trained feature extractor model.
        number_of_batches (int): The number of steps (batches) to process.

    Returns:
        tuple: A tuple containing:
            - features (numpy.ndarray): Extracted features for all samples in the dataset.
            - labels (numpy.ndarray): Corresponding labels for the samples.
    """
    features = []
    labels = []
    for _ in range(number_of_batches):
        batch_x, batch_y = next(generator)
        batch_features = model.predict(batch_x, verbose=0)
        features.append(batch_features)
        labels.append(batch_y)
    features = np.vstack(features)[:generator.samples]  
    labels = np.hstack(labels)[:generator.samples]
    return features, labels

def train_and_evaluate_model(model_name, preprocess_function):
    print(f"\nTraining with {model_name}...")
    # Set image size based on model
    if model_name == 'Xception':
        img_h, img_w = 299, 299
    else:
        img_h, img_w = 224, 224
        
    # Create ImageDataGenerators
    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_function)
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_function)
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_function)
    
    train_dir = os.path.join(split_dataset_dir, "train")
    val_dir = os.path.join(split_dataset_dir, "validation")
    test_dir = os.path.join(split_dataset_dir, "test")
    
    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_h, img_w), batch_size=batch_size, class_mode='binary', shuffle=True, seed=42)
    val_generator = val_datagen.flow_from_directory(val_dir, target_size=(img_h, img_w), batch_size=batch_size, class_mode='binary', shuffle=False, seed=42)
    test_generator = test_datagen.flow_from_directory(test_dir, target_size=(img_h, img_w), batch_size=batch_size, class_mode='binary', shuffle=False, seed=42)
    
    class_indices = test_generator.class_indices    
    
    # Load the pre-trained model
    if model_name == 'VGG19':
        Pretrained_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'ResNet50':
        Pretrained_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'VGG16':
        Pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'MobileNetV2':
        Pretrained_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'Xception':
        Pretrained_model = Xception(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'EfficientNetB0':
        Pretrained_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'DenseNet121':
        Pretrained_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    else:
        raise ValueError("Invalid model name")
        
    # Freeze the pre-trained layers
    Pretrained_model.trainable = False
    
    # Create feature extractor
    feature_extractor = Model(inputs=Pretrained_model.input, outputs=GlobalAveragePooling2D()(Pretrained_model.output))

    # Calculate steps for feature extraction
    train_batchs_number = math.ceil(train_generator.samples / batch_size)
    val_batchs_number = math.ceil(val_generator.samples / batch_size)
    test_batchs_number = math.ceil(test_generator.samples / batch_size)
    
    # Extract features
    print("Extracting features for training set...")
    train_features, train_labels = extract_features(train_generator, feature_extractor, train_batchs_number)
    print("Extracting features for validation set...")
    val_features, val_labels = extract_features(val_generator, feature_extractor, val_batchs_number)
    print("Extracting features for test set...")
    test_features, test_labels = extract_features(test_generator, feature_extractor, test_batchs_number)
    
    # Define simplified custom head
    feature_dim = feature_extractor.output_shape[-1]
    input_layer = Input(shape=(feature_dim,))
    x = Dense(512)(input_layer)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    predictions = Dense(1, activation='sigmoid', dtype='float32')(x)
    custom_model = Model(inputs=input_layer, outputs=predictions)
    
    # Compile the custom head model
    custom_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    
    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Train the custom head
    history = custom_model.fit(train_features, train_labels, batch_size=batch_size, epochs=epochs,
                            validation_data=(val_features, val_labels), callbacks=[early_stopping])
    
    # Create full model for efficiency analysis
    full_model = Model(inputs=feature_extractor.input, outputs=custom_model(feature_extractor.output))
    # Efficiency Analysis
    test_generator.reset()
    batch_x, batch_y = next(test_generator)
    batch_size_efficiency = len(batch_x)
    
    # Inference Speed
    start_time = time.time()
    _ = full_model.predict(batch_x, verbose=0)
    inference_time = time.time() - start_time
    inference_speed = inference_time / batch_size_efficiency
    
    # FLOPs
    try:
        
        flops = get_flops(full_model)
        flops_million = flops / 1e6
    except Exception as eer :
        flops_million = "N/A"
        print(f"\n\nFLOPs Error : {eer}\n\n")
    
    # Inside the efficiency analysis section of train_and_evaluate_model:

    # Memory Consumption (parameters + activations)
    param_count = full_model.count_params()
    param_memory = param_count * 4  # bytes (float32)
    
    activation_memory = 0
    for layer in full_model.layers:
        # Handle InputLayer separately
        if isinstance(layer, InputLayer):
            output_shape = full_model.input_shape  # Get from model's input shape
        else:
            # Use Keras backend to get output shape safely
            output_shape = K.int_shape(layer.output)
        
        # Handle multi-output layers
        if isinstance(output_shape, list):
            output_shape = output_shape[0]
        
        # Process dimensions (skip batch dim and handle dynamic shapes)
        activation_dims = [dim for dim in output_shape[1:] if dim is not None]
        
        # Calculate activation size
        layer_activation_size = 1
        for dim in activation_dims:
            layer_activation_size *= dim
        
        activation_memory += layer_activation_size
    
    activation_memory *= 4  # bytes (float32)
    total_memory = param_memory + activation_memory  # bytes
    

    # Model Size
    model_size = param_memory
    
    # Energy Consumption
    energy_consumption = "N/A"
    try:
        
        from pyJoules.handler.csv_handler import CSVHandler
        from pyJoules.energy_meter import measure_energy

        energy_csv = os.path.join(run_dir, f"{model_name}_energy.csv")
        csv_handler = CSVHandler(energy_csv)
    
        @measure_energy(handler=csv_handler)
        def run_inference(model, data):
            return model.predict(data, verbose=0)
    
        # Execute the decorated function
        run_inference(full_model, batch_x)
    
        # Save the recorded data
        csv_handler.save_data()
        # Read and sum the energy consumption
        import pandas as pd
        df = pd.read_csv(energy_csv,sep=';')
        energy_consumption = df['nvidia_gpu_0'].sum()
    except Exception as e:
        print(f"Energy consumption error: {e}")
    
    # Evaluate on test set
    test_loss, test_accuracy = custom_model.evaluate(test_features, test_labels, verbose=1)
    print(f"\n{model_name} Test Loss: {test_loss:.4f}")
    print(f"{model_name} Test Accuracy: {test_accuracy:.4f}")
    
    # Make predictions
    predictions = custom_model.predict(test_features, verbose=1)
    y_pred = (predictions > 0.5).astype(int).flatten()
    y_true = test_labels
    
    # Compute performance metrics
    cm = confusion_matrix(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='binary')
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    
    # Compute ROC curve and AUC
    fpr, tpr, thresholds = roc_curve(y_true, predictions)
    roc_auc = roc_auc_score(y_true, predictions)
    
    return {
        'model': custom_model,
        'history': history.history,
        'confusion_matrix': cm,
        'roc': (fpr, tpr, roc_auc) if 'fpr' in locals() else (None, None, None),
        'performance': {'accuracy': test_accuracy, 'f1': f1, 'precision': precision, 'recall': recall},
        'labels': list(class_indices.keys()),
        'efficiency': {
            'inference_speed': inference_speed,
            'flops_million': flops_million,
            'memory_consumption_bytes': total_memory,
            'model_size_bytes': model_size,
            'energy_consumption_joules': energy_consumption
        }
    }

def plot_results(results, run_dir):
    model_names = list(results.keys())
    num_models = len(model_names)
    
    # 1. Plot Confusion Matrices
    fig_cm, axes_cm = plt.subplots(1, num_models, figsize=(5*num_models, 4))
    if num_models == 1:
        axes_cm = [axes_cm]
    for ax, name in zip(axes_cm, model_names):
        cm = results[name]['confusion_matrix']
        im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        ax.set_title(f"{name} Confusion Matrix")
        tick_marks = np.arange(len(results[name]['labels']))
        ax.set_xticks(tick_marks)
        ax.set_xticklabels(results[name]['labels'], rotation=45)
        ax.set_yticks(tick_marks)
        ax.set_yticklabels(results[name]['labels'])
        thresh = cm.max() / 2.0
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, format(cm[i, j], 'd'),
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black")
        fig_cm.colorbar(im, ax=ax)
    fig_cm.tight_layout(rect=[0, 0, 1, 0.95])
    fig_cm.suptitle("Confusion Matrices", fontsize=16)
    plt.savefig(os.path.join(run_dir, "confusion_matrices.png"))
    plt.close(fig_cm)
    
    # 2. Plot ROC Curves
    fig_roc, axes_roc = plt.subplots(1, num_models, figsize=(5*num_models, 4))
    if num_models == 1:
        axes_roc = [axes_roc]
    for ax, name in zip(axes_roc, model_names):
        # Get ROC data with error handling
        roc_data = results[name].get('roc', (None, None, None))
        fpr, tpr, roc_auc = roc_data
        
        # Handle missing or invalid ROC data
        if fpr is None or tpr is None or roc_auc is None:
            print(f"Warning: Missing ROC data for {name}")
            ax.text(0.5, 0.5, 'No valid ROC data', ha='center', va='center')
            ax.set_title(f"ROC Curve - {name}")
            continue
        
        # Plot with formatted AUC
        label = f"AUC = {roc_auc:.2f}" if isinstance(roc_auc, (int, float)) else "AUC = N/A"
        ax.plot(fpr, tpr, lw=2, label=label)
        ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title(f"ROC Curve - {name}")
        ax.legend(loc="lower right")
    fig_roc.tight_layout(rect=[0, 0, 1, 0.95])
    fig_roc.suptitle("ROC Curves", fontsize=16)
    plt.savefig(os.path.join(run_dir, "roc_curves.png"))
    plt.close(fig_roc)
    
    # 3. Plot Accuracy and Loss Curves for each model
    for name in model_names:
        history = results[name]['history']
        epochs_range = range(1, len(history['accuracy']) + 1)
        fig_model, (ax_acc, ax_loss) = plt.subplots(1, 2, figsize=(12, 5))
        ax_acc.plot(epochs_range, history['accuracy'], marker='o', label='Train Accuracy')
        ax_acc.plot(epochs_range, history['val_accuracy'], marker='x', linestyle='--', label='Validation Accuracy')
        ax_acc.set_title(f"{name} Accuracy")
        ax_acc.set_xlabel('Epoch')
        ax_acc.set_ylabel('Accuracy')
        ax_acc.legend()
        
        ax_loss.plot(epochs_range, history['loss'], marker='o', label='Train Loss')
        ax_loss.plot(epochs_range, history['val_loss'], marker='x', linestyle='--', label='Validation Loss')
        ax_loss.set_title(f"{name} Loss")
        ax_loss.set_xlabel('Epoch')
        ax_loss.set_ylabel('Loss')
        ax_loss.legend()
        
        fig_model.suptitle(f"Accuracy and Loss Curves - {name}", fontsize=16)
        fig_model.tight_layout(rect=[0, 0, 1, 0.93])
        plt.savefig(os.path.join(run_dir, f"{name}_training_curves.png"))
        plt.close(fig_model)
    
    # 4. Overall Performance Comparison Table
    col_labels = ["Model", "Accuracy", "Precision", "Recall", "F1 Score"]
    cell_text = []
    for name in model_names:
        perf = results[name]['performance']
        row = [name,
               f"{perf['accuracy']:.4f}",
               f"{perf['precision']:.4f}",
               f"{perf['recall']:.4f}",
               f"{perf['f1']:.4f}"]
        cell_text.append(row)
    
    fig_table, ax_table = plt.subplots(figsize=(8, len(model_names)*0.8+1))
    ax_table.axis('tight')
    ax_table.axis('off')
    table = ax_table.table(cellText=cell_text, colLabels=col_labels, loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1, 2)
    fig_table.suptitle("Overall Performance Comparison", fontsize=16)
    plt.savefig(os.path.join(run_dir, "performance_table.png"))
    plt.close(fig_table)
    
    # 5. Save training histories to pickle files
    for name in model_names:
        history = results[name]['history']
        with open(os.path.join(run_dir, f"{name}_history.pkl"), "wb") as f:
            pickle.dump(history, f)
    
    # 6. Efficiency Comparison Table
    col_labels_eff = ["Model", "Inference Speed (s/img)", "FLOPs (MFLOPs)", "Memory (MB)", "Model Size (MB)", "Energy (J)"]
    cell_text_eff = []
    for name in model_names:
        eff = results[name]['efficiency']
        row = [
            name,
            f"{eff['inference_speed']:.4f}",
            f"{eff['flops_million']:.2f}" if eff['flops_million'] != 'N/A' else 'N/A',
            f"{eff['memory_consumption_bytes']/1e6:.2f}",
            f"{eff['model_size_bytes']/1e6:.2f}",
            f"{eff['energy_consumption_joules']:.2f}" if isinstance(eff['energy_consumption_joules'], float) else eff['energy_consumption_joules']
        ]
        cell_text_eff.append(row)
    
    fig_table_eff, ax_table_eff = plt.subplots(figsize=(12, len(model_names)*0.8+1))
    ax_table_eff.axis('tight')
    ax_table_eff.axis('off')
    table_eff = ax_table_eff.table(cellText=cell_text_eff, colLabels=col_labels_eff, loc='center')
    table_eff.auto_set_font_size(False)
    table_eff.set_fontsize(12)
    table_eff.scale(1, 2)
    fig_table_eff.suptitle("Efficiency Comparison", fontsize=16)
    plt.savefig(os.path.join(run_dir, "efficiency_table.png"))
    plt.close(fig_table_eff)

    
    print("Training histories saved for each model.")




# Update the main execution block to use the new function as is
if __name__ == "__main__":
    Base_Folder = 'D:/Learning/University of sadat/Grade 4/Semester 2/06- Graduation Project/Coding/' 
    split_dataset_dir = f'{Base_Folder}00- The DataSet/Dataset_split_Before_Preprocessing'
    
    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    run_dir = os.path.join(f'{Base_Folder}runs_codes', f"{current_time}_Before_preprocess")
    os.makedirs(run_dir, exist_ok=True)

    batch_size = 32
    epochs = 20
    
    results = {}
    models = {
        'VGG19': vgg19_preprocess,
        'ResNet50': resnet50_preprocess,
        'VGG16': vgg16_preprocess,
        'MobileNetV2': mobilenetv2_preprocess,
        'Xception': xception_preprocess,
        'EfficientNetB0': efficientnetb0_preprocess,
        'DenseNet121': densenet121_preprocess
    }
    
    for model_name, preprocess in models.items():
        results[model_name] = train_and_evaluate_model(model_name, preprocess)

        # Print efficiency metrics
        eff = results[model_name]['efficiency']
        print(f"\n{model_name} Efficiency Metrics:")
        print(f"Inference Speed: {eff['inference_speed']:.4f} seconds/image")
        print(f"FLOPs: {eff['flops_million']:.2f} MFLOPs" 
              if isinstance(eff['flops_million'], (int, float)) 
              else f"FLOPs: {eff['flops_million']} MFLOPs")
        print(f"Memory Consumption: {eff['memory_consumption_bytes']/1e6:.2f} MB")
        print(f"Model Size: {eff['model_size_bytes']/1e6:.2f} MB")
        print(f"Energy Consumption: {eff['energy_consumption_joules']:.2f} J" if isinstance(eff['energy_consumption_joules'], (int, float)) else f"Energy Consumption: {eff['energy_consumption_joules']} J")
        # Save the custom head model instead
        results[model_name]['model'].save(os.path.join(run_dir, f"{model_name}_custom_model.h5"))
        print(f"{model_name} custom head model saved.")
    
    plot_results(results, run_dir)
    print(f"All outputs saved to directory: {run_dir}")



Training with VGG19...
Found 35 images belonging to 2 classes.
Found 3 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Extracting features for training set...
Extracting features for validation set...
Extracting features for test set...
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 210ms/step - accuracy: 0.4696 - loss: 0.7689 - val_accuracy: 0.6667 - val_loss: 0.7026
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.6967 - loss: 0.5295 - val_accuracy: 0.6667 - val_loss: 0.6753
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5979 - loss: 0.6146 - val_accuracy: 0.6667 - val_loss: 0.6426
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.7054 - loss: 0.4709 - val_accuracy: 0.6667 - val_loss: 0.6156
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.8336 - loss: 0.4740 - val_accuracy: 0.6667 - val_loss: 0.6052
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.7937 - loss: 0.4916 - val_accuracy: 0.6667 - val_lo

d:\Learning\University of sadat\Grade 4\Semester 2\06- Graduation Project\Ai_Env\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_1223
Received: inputs=['Tensor(shape=(1, 224, 224, 3))']
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.6667 - loss: 0.6945

VGG19 Test Loss: 0.6945
VGG19 Test Accuracy: 0.6667
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step



VGG19 Efficiency Metrics:
Inference Speed: 0.2398 seconds/image
FLOPs: 39038.39 MFLOPs
Memory Consumption: 147.29 MB
Model Size: 81.16 MB
Energy Consumption: 16401 J
VGG19 custom head model saved.

Training with ResNet50...
Found 35 images belonging to 2 classes.
Found 3 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Extracting features for training set...
Extracting features for validation set...
Extracting features for test set...
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 265ms/step - accuracy: 0.4315 - loss: 1.0289 - val_accuracy: 0.3333 - val_loss: 1.0332
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.6188 - loss: 0.7144 - val_accuracy: 0.3333 - val_loss: 0.9585
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.5494 - loss: 0.6915 - val_accuracy: 0.3333 - val_loss: 0.9163
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.6967 - loss: 0.5239 - val_accuracy: 0.3333 - val_loss: 0.8684
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s

d:\Learning\University of sadat\Grade 4\Semester 2\06- Graduation Project\Ai_Env\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_1253
Received: inputs=['Tensor(shape=(1, 224, 224, 3))']
  warnings.warn(msg)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6667 - loss: 0.6276

EfficientNetB0 Test Loss: 0.6276
EfficientNetB0 Test Accuracy: 0.6667
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step



EfficientNetB0 Efficiency Metrics:
Inference Speed: 0.3014 seconds/image
FLOPs: 802.10 MFLOPs
Memory Consumption: 119.89 MB
Model Size: 18.83 MB
Energy Consumption: 3008 J
EfficientNetB0 custom head model saved.

Training with DenseNet121...
Found 35 images belonging to 2 classes.
Found 3 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Extracting features for training set...
Extracting features for validation set...
Extracting features for test set...
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step - accuracy: 0.2652 - loss: 0.9980 - val_accuracy: 0.6667 - val_loss: 0.6269
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.3726 - loss: 0.8854 - val_accuracy: 0.6667 - val_loss: 0.6125
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.4610 - loss: 0.8284 - val_accuracy: 0.6667 - val_loss: 0.5978
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.4506 - loss: 0.7815 - val_accuracy: 0.6667 - val_loss: 0.5834
Epoch 5/20
2/2 ━━━━━

d:\Learning\University of sadat\Grade 4\Semester 2\06- Graduation Project\Ai_Env\Lib\site-packages\keras\src\models\functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor_2011
Received: inputs=['Tensor(shape=(1, 224, 224, 3))']
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.6667 - loss: 0.6037

DenseNet121 Test Loss: 0.6037
DenseNet121 Test Accuracy: 0.6667
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step



DenseNet121 Efficiency Metrics:
Inference Speed: 0.5274 seconds/image
FLOPs: 5701.47 MFLOPs
Memory Consumption: 229.96 MB
Model Size: 30.26 MB
Energy Consumption: 7670 J
DenseNet121 custom head model saved.
Training histories saved for each model.
All outputs saved to directory: D:/Learning/University of sadat/Grade 4/Semester 2/06- Graduation Project/Coding/runs_codes\2025-04-21_17-25-55_Before_preprocess
